In [ ]:
# Подключение модулей
from keras.models import Sequential
from keras.layers.core import Activation, Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.datasets import cifar10
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
import cv2, numpy as np
import keras

In [ ]:
#загрузили данные
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [ ]:
# Размер изображений
img_width, img_height = 32, 32
# Размер мини-выборки
batch_size = 32
# Кол-во изображений для обучения
nb_train_samples = 50000
# Кол-во изображений для теста
nb_test_samples = 10000

In [ ]:
# normalize inputs from 0-255 to 0.0-1.0
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train = x_train / 255.0
x_test = x_test / 255.0

# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

In [ ]:
# Создание экземпляра модели сети VGG16
vgg16_net = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
# weights - веса предварительно обученной сети
# include_top = false означает, что мы загружаем только сверточную часть сети, без квалификационной
# input_shape - размер тензора

# Сверточная часть сети
vgg16_net.trainable = True
trainable = False
for layer in vgg16_net.layers:
  if layer.name == 'block5_conv1':
    trainable = True
  layer.trainable = trainable

vgg16_net.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0     

In [ ]:
# Создание модели составной сети
model = Sequential()
# Добавляем сверточные слои
model.add(vgg16_net)
# Преобразуем двумерный массив vgg16 в одномерный
model.add(Flatten())
# Полносвязный слой
model.add(Dense(256, activation='relu'))
# Слой регуляризации (для предотвращения переобучения)
model.add(Dropout(0.5))
# Кол-во классов
model.add(Dense(num_classes, activation='sigmoid'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 1, 1, 512)         14714688  
_________________________________________________________________
flatten_2 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                2570      
Total params: 14,848,586
Trainable params: 7,213,322
Non-trainable params: 7,635,264
_________________________________________________________________


In [ ]:
# Компилируем составную сеть
epochs = 5
lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd, 
              metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
# Обучение составной сети
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs = 5, batch_size=batch_size)


Epoch 1/5
1563/1563 [==============================] - 1254s 801ms/step - loss: 1.4192 - accuracy: 0.5033 - val_loss: 0.8768 - val_accuracy: 0.6947
Epoch 2/5
1563/1563 [==============================] - 1314s 841ms/step - loss: 0.8624 - accuracy: 0.7055 - val_loss: 0.7972 - val_accuracy: 0.7256
Epoch 3/5
1563/1563 [==============================] - 1348s 863ms/step - loss: 0.7308 - accuracy: 0.7505 - val_loss: 0.7585 - val_accuracy: 0.7423
Epoch 4/5
1563/1563 [==============================] - 1359s 869ms/step - loss: 0.6421 - accuracy: 0.7782 - val_loss: 0.7378 - val_accuracy: 0.7457
Epoch 5/5
1563/1563 [==============================] - 1363s 872ms/step - loss: 0.5889 - accuracy: 0.7958 - val_loss: 0.7437 - val_accuracy: 0.7476


In [ ]:
# Final evaluation of the model
scores = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 74.76%
